In [3]:
# Core TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical


In [4]:
# Load CIFAR-10 dataset
# CIFAR-10 contains 60,000 32x32 RGB images across 10 classes
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Convert class labels to one-hot encoded vectors
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Preprocessing function:
# - Resize images to 224x224 to match EfficientNet input requirements
# - Normalize pixel values to [0, 1]
def preprocess(image, label):
    image = tf.image.resize(image, (224,224))
    image = image / 255.0
    return image, label

# Create TensorFlow Dataset for efficient input pipeline

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [5]:
# Baseline CNN model trained from scratch (no pre-trained knowledge)
# Acts as a performance reference point
baseline_model = models.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

baseline_history = baseline_model.fit(
    train_ds,  # Use the preprocessed dataset
    epochs=5,
    validation_data=test_ds # Use the preprocessed test dataset for validation
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 92s 54ms/step - accuracy: 0.3875 - loss: 1.8108 - val_accuracy: 0.5117 - val_loss: 1.3591
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.5357 - loss: 1.3074 - val_accuracy: 0.5366 - val_loss: 1.3064
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 51ms/step - accuracy: 0.6047 - loss: 1.1115 - val_accuracy: 0.5747 - val_loss: 1.2033
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 51ms/step - accuracy: 0.6668 - loss: 0.9410 - val_accuracy: 0.5665 - val_loss: 1.3092
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.7292 - loss: 0.7645 - val_accuracy: 0.5675 - val_loss: 1.4698


In [ ]:
# Load EfficientNetB0 pre-trained on ImageNet
# include_top=False removes the original ImageNet classifier
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

# Freeze all layers of the pre-trained model
# This prevents updating ImageNet-learned weights
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train only the newly added classifier layers
history = model.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds
)


In [ ]:
# Unfreeze the pre-trained EfficientNet layers
# Allows full network to adapt to CIFAR-10 domain
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Recompile with a very small learning rate
# This avoids destroying pre-trained features (catastrophic forgetting)
fine_tune_history =model.fit(
    train_ds,
    epochs=3,
    validation_data=test_ds
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.plot(baseline_history.history['accuracy'], label='Baseline Train')
plt.plot(baseline_history.history['val_accuracy'], label='Baseline Val')

plt.plot(history.history['accuracy'], label='TL Train (Frozen)')
plt.plot(history.history['val_accuracy'], label='TL Val (Frozen)')

plt.plot(
    range(len(history.history['accuracy']),
          len(history.history['accuracy']) + len(fine_tune_history.history['accuracy'])),
    fine_tune_history.history['accuracy'],
    label='TL Train (Fine-tuned)'
)

plt.title("Accuracy Comparison: Baseline vs Transfer Learning")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
baseline_test_acc = baseline_model.evaluate(test_ds, verbose=0)[1]
tl_test_acc = model.evaluate(test_ds, verbose=0)[1]

plt.figure(figsize=(6,4))
plt.bar(['Baseline CNN', 'Transfer Learning'],
        [baseline_test_acc, tl_test_acc])
plt.title("Test Accuracy Comparison")
plt.ylabel("Accuracy")
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataset, title):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        preds = model.predict(images, verbose=0)
        y_pred.extend(np.argmax(preds, axis=1))
        y_true.extend(np.argmax(labels.numpy(), axis=1))

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(title)
    plt.show()

plot_confusion_matrix(baseline_model, test_ds, "Baseline CNN Confusion Matrix")
plot_confusion_matrix(model, test_ds, "Transfer Learning Confusion Matrix")


In [ ]:
### Key Observation
- Feature extraction alone resulted in low accuracy due to domain mismatch
- Fine-tuning enabled EfficientNet to adapt its representations to CIFAR-10
- Demonstrates the importance of controlled fine-tuning in transfer learning
